# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Описание данных

- **Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.
- **Целевой признак**: количество страховых выплат клиенту за последние 5 лет.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

RANDOM_STATE = 123

## Загрузка данных

In [2]:
df = pd.read_csv('../datasets/insurance.csv')
df.info()
print('\nКоличество дублей', df.duplicated().sum())
df.sample(5, random_state=RANDOM_STATE)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB

Количество дублей 153


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2648,1,29.0,39300.0,0,0
2456,1,32.0,37800.0,1,0
4557,1,41.0,54000.0,4,0
4884,0,34.0,40200.0,0,0
92,1,25.0,26700.0,1,0


Переименуем столбцы

In [3]:
df.columns = ['gender', 'age', 'salary', 'relatives', 'insurance']
df.head(3)

,gender,age,salary,relatives,insurance
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


Дубли удалим, вряд ли все значения сразу совпали случайно. Тип данных всех полей можно поменять на целочисленный

In [4]:
df = df.drop_duplicates().reset_index(drop=True)
df = df.astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   gender     4847 non-null   int32
 1   age        4847 non-null   int32
 2   salary     4847 non-null   int32
 3   relatives  4847 non-null   int32
 4   insurance  4847 non-null   int32
dtypes: int32(5)
memory usage: 94.8 KB


Уникальные значения полей

In [5]:
print('Уникальные значения')
for col in ['gender', 'relatives', 'insurance', 'age']:
    print(col.rjust(10), ':', np.sort(df[col].unique()))

Уникальные значения
    gender : [0 1]
 relatives : [0 1 2 3 4 5 6]
 insurance : [0 1 2 3 4 5]
       age : [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 65]


В данных не обнаружено аномалий, пропусков.

## Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится

**Обоснование:** 

Для ответа на вопрос получим выражения для весов линейной регрессии и ответов на ней после проеобразования признаков. Чтобы качество модели не изменилось, необходимо, что бы "новые" ответы совпадали со "старыми"  

Рассмотрим в векторном виде задачу линейной регрессии. 

Выражение для предсказаний $a$ имеет вид:
$$ a = X\omega $$ 
где:
- $X$ — матрица признаков (нулевой столбец состоит из единиц)
- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

В основе регрессионного анализа лежит метод наименьших квадратов, а именно минимизация с его помощью квадратов ошибки между аппроксимационной зависимостью и ответами $y$ (вектором целевого признака). То есть: 
$$(X\omega-y)^T(X\omega-y) \rightarrow \min_{\omega}$$
Или через квадратичную функцию потерь:
$$ w = \arg\min_w MSE(X\omega, y) $$
Продифференцировав предыдущее выражение по параметру $\omega$ и приравняв производные к нулю получим решение для задачи минимизации. 
Тогда формула обучения:
$$ w = (X^T X)^{-1} X^T y $$


Рассмотрим теперь матрицу преобразованных признаков $X'$, которая получается умножением на обратимую матрицу
$$X' = XP$$
где $P$ — матрица, на которую умножаются признаки

Вектор предсказаний для преобразованных признаков
$$ a' = X'\omega' $$ 
Вектор весов:
$$\omega' = (X'^T X')^{-1} X'^T y$$
Подставим в это выражение $X'$. Пользуясь свойствами умножения матриц, обратных матриц и транспонирования, получим:
$$\omega'  = ((XP)^T XP)^{-1} (XP)^T y = ((P^T (X^T X) P)^{-1} P^T X^T y \\ 
           = P^{-1} (X^T X)^{-1} (P^T)^{-1}  P^T X^T y = P^{-1} (X^T X)^{-1} X^T y = P^{-1} \omega$$
Таким образом, "новый" вектор весов отличается от "старого" умножением на обратную матрицу слева. Выразим теперь $a'$ через признаки и веса до преобразования
$$ a' = X'\omega' = X P P^{-1} \omega = X \omega = a$$ 
Что и требовалось доказать

## Алгоритм преобразования

Для преобразования входных данных с целью защиты личных данных пользователей применим умножение на обратимую матрицу, значения для которой сгенерируем случайным образом.

**Алгоритм**

0. Вводим матрицу признаков $X$
1. Генерация кватратной матрицы $P$ со случайными значениями, размерность матрицы - по количеству признаков.
2. Проверка матрицы $P$ на обратимость.
3. Получение матрицы преобразованных признаков $X'=XP$

Реализуем описанный алгоритм в виде функции

- Также возможно преобразование данных умножением матрицы на постоянный множитель, добавление постоянного числа

**Обоснование**

Матрица преобразования должна быть обратимой, следовательно она должна быть квадратной и невырожденной (её детерминант должен быть отличен от нуля)

Если, количество признаков $= n$, а количество объектов $= m$, то размер матрицы преобразования должен быть $n \times n$. Для обратимости матрица должна быть квадратной.

В таком случае при умножении исходной матрицы признаков размером $m \times n$ на матрицу преобразования $n \times n$ получим новую матрицу признаков но исходного размера $m \times n$. $$X_{m \times n} \cdot P_{n \times n} = X'_{m \times n}$$

При этом очевидно, что такое проеобразование не "перемешивает" объекты в выборке. Индексация объектов сохранится, поскольку признаки одного объекта умножаются на каждый столбец матрицы преобразования, формируя объект с новыми признаками. По этой причине преобразование можно применять не ко всей выборке сразу, а по частям: преобразованные признаки в обоих случаях останутся одинаковыми (при условии одинакового набора признаков) 

Покажем, что умножение на число не меняет вектор предсказаний
$$\omega'  = ((b\cdot X)^T (b\cdot X))^{-1} (b\cdot X)^T y = (b^2\cdot X^T X)^{-1} b\cdot X^T y \\ 
           = \frac{1}{b^2}\cdot b \cdot(X^T X)^{-1} X^T y = \frac{1}{b}\cdot (X^T X)^{-1} X^T y = \frac{1}{b}\cdot \omega$$
$$ a' = X'\omega' = b\cdot X \cdot\frac{1}{b} \cdot \omega = X \omega = a$$ 

Смещение же всех признаков на постоянное число вовсе не поменяет коэффициентов линейной регрессии

In [6]:
def transform_features(features, transformation_matrix=None, random_state=None):
    ''' Преобразование матрицы признаков features умножением на обратимую матрицу. 
    Обратимая матрица может быть сгененирована фунцией или передана пользователем. 
    Возвращает преобразованную матрицу исходной рамерности
    
    Параметры :
    ----
    `features` : np.array / pd.DataFrame - массив признаков
    `transformation_matrix` : np.array - матрица преобразования, обе размерности должны совпадать с количеством признаков в features
    `random_state` : int - случайное состояние numpy

    Возвращает :
    ----
    Для случая `transformation_matrix = None` :
        `tuple` : `(np.array, np.array)` - (признаки, матрица преобразования)
    Для остальных случаев :
        : `np.array` признаков
    '''
       
    n = features.shape[1]

    if transformation_matrix is None:
        np.random.seed(random_state)
        det = 0
        while det == 0:
            transformation_matrix = np.random.normal(size=(n,n))
            det = np.linalg.det(transformation_matrix)
        return (features @ transformation_matrix, transformation_matrix) 

    else:
        if (transformation_matrix.shape[0] != n or 
            transformation_matrix.shape[1] != n):
            print('Размерность матрицы преобразования {} не соответсвует размерности признаков ({})'
                  .format(transformation_matrix.shape, n))
            return None
        if np.linalg.det(transformation_matrix) == 0:
            print('Матрицы преобразования необратима! (det = 0)')
            return None
        return features @ transformation_matrix

Посмотрим, как изменятся признаки в исходном датасете

In [7]:
transform_features(df[df.columns[:-1]])[0].sample(5, random_state=RANDOM_STATE)

,0,1,2,3
1785,118380.986564,-11269.642477,28088.639189,80067.623337
193,111957.619491,-10656.630332,26565.265523,75725.044739
4358,110144.218085,-10471.453349,26144.868311,74509.828760
549,122743.408716,-11682.384135,29126.118667,83019.293588
2959,79582.603237,-7559.418567,18895.375553,53841.993868


Признаки преобразованы

## Проверка алгоритма

Подготовим признаки. 
- Разделим выборку на обучающую и тестовую
- Сформируем преобразованные обучающую и тестовую выборки

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    df[df.columns[:-1]].values, df[df.columns[-1]].values, 
    test_size=0.25, random_state=RANDOM_STATE)

X_train_new, transformation_matrix = transform_features(X_train, random_state=RANDOM_STATE)
X_test_new = transform_features(X_test, transformation_matrix=transformation_matrix, random_state=RANDOM_STATE)

print('Матрица преобразования:'),
print(transformation_matrix)

Матрица преобразования:
[[-1.0856306   0.99734545  0.2829785  -1.50629471]
 [-0.57860025  1.65143654 -2.42667924 -0.42891263]
 [ 1.26593626 -0.8667404  -0.67888615 -0.09470897]
 [ 1.49138963 -0.638902   -0.44398196 -0.43435128]]


Проверим, как изменется R2 метрика и значения весов линейной регрессии при преобразовании признаков. Рассмотрим отдельно разные случаи

In [9]:
def check_lr_model(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    r2 = model.score(X_test, y_test)
    print('w  = {} \nR2 = {:.12f}'.format(model.coef_, r2))
    return (r2, model.coef_)

**Без преобразования**

In [10]:
_, w = check_lr_model(X_train, X_test, y_train, y_test);

w  = [ 1.19844319e-02  3.67531733e-02 -1.94830996e-08 -1.50310819e-02] 
R2 = 0.409789581465


**Умножение на число и смещение**

In [11]:
mult = np.random.normal(scale=100)
shift = np.random.normal(scale=1000)

print('Смещение значений всех признаков на', shift)
_, w_s = check_lr_model(X_train+shift, X_test+shift, y_train, y_test)
print('\nУмножение всех признаков на {:.4f} и смещение на {:.4f}'.format(mult, shift))
_, w_ms = check_lr_model(X_train*mult+shift, X_test*mult+shift, y_train, y_test)

print('\nОтношения весов:', w_s / w_ms)

Смещение значений всех признаков на 2186.7860889737867
w  = [ 1.19844319e-02  3.67531733e-02 -1.94830996e-08 -1.50310819e-02] 
R2 = 0.409789581465

Умножение всех признаков на 220.5930 и смещение на 2186.7861
w  = [ 5.43282493e-05  1.66610781e-04 -8.83214736e-11 -6.81394303e-05] 
R2 = 0.409789581465

Отношения весов: [220.59300827 220.59300827 220.59300839 220.59300827]


R2 мера не поменялась.

В случае, когда мы сместили значения всех признаков на одинаковую величину веса также не изменились.

При умножении веса изменились в `mult` раз

**Умножение на обратимую матрицу**

In [12]:
check_lr_model(X_train_new, X_test_new, y_train, y_test);

w  = [-0.0291558  -0.02372427 -0.02310321 -0.00699133] 
R2 = 0.409789581465


In [13]:
mult = np.random.normal(scale=100)
shift = np.random.normal(scale=1000)
print('Умножение уже преобразованных признаков на {:.4f} и смещение на {:.4f}'.format(mult, shift))
check_lr_model(X_train_new*mult+shift, X_test_new*mult+shift, y_train, y_test);

Умножение уже преобразованных признаков на 100.4054 и смещение на 386.1864
w  = [-2.90380857e-04 -2.36284795e-04 -2.30099262e-04 -6.96310004e-05] 
R2 = 0.409789581465


Качество не поменялось

## Вывод

Исследовано влияние преобразования данных путём умножения признаков на обратимую матрицу, на число, смещение признаков.  Проверено качество обучения линейной регрессии (на метрике R2) для исходных данных и преобразованых.
Во всех случаях качество обучения осталось одинаковым. Заметим, что разница занчений `R2` в 12 и далее знаках после запятой обусловлена округлениями вещественных чисел в numpy и python в процессе вычислений.